<a href="https://colab.research.google.com/github/victoreduardo/fl_experiment_with_airtlab/blob/main/Paper_FL_%2B_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mlflow
!pip install flwr

In [ ]:
from typing import Dict, Optional, Tuple

import flwr as fl
import tensorflow as tf
import os
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing import image

import mlflow

# Metricas
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
MODELS_PATH = '/content/drive/MyDrive/FL/'
SAVE_MODELS_PATH = '/content/drive/MyDrive/FL/model'
DATASET_PATH = '/content/drive/MyDrive/FL/data/AIRTLab/3_clients/'
DATASET_VALID_PATH = '/content/drive/MyDrive/FL/data/AIRTLab/test/'
TARGET_SIZE = (120,160)

VIOLENCIA_DIR = 'violent'
N_VIOLENCIA_DIR = 'non-violent'

In [ ]:
!databricks configure --host https://community.cloud.databricks.com/
mlflow.set_tracking_uri("databricks")
mlflow.set_experiment("/Users/vess@cesar.school/FL-AIRTLab")

Username: vess@cesar.school
Password: 
Repeat for confirmation: 


<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2852776061936707', experiment_id='2852776061936707', lifecycle_stage='active', name='/Users/vess@cesar.school/FL-AIRTLab', tags={'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'vess@cesar.school',
 'mlflow.ownerId': '8444989453852335'}>

In [ ]:
mlflow.end_run()
mlflow.start_run(run_name='load_mobilenetv2')

<ActiveRun: >

### FL Utils

In [ ]:
ROUND_NUM = 1

In [ ]:
def get_eval_fn(model, accuracies):
    """Return an evaluation function for server-side evaluation."""

    # Load data and model here to avoid the overhead of doing it in `evaluate` itself
    evoluation_loader = evoluation_dataset()

    # The `evaluate` function will be called after every round
    def evaluate(
            weights: fl.common.Weights,
    ) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
        model.set_weights(weights)  # Update model with the latest parameters
        loss, accuracy = model.evaluate(evoluation_loader)
        
        accuracies.append(accuracy)
        round_num = len(accuracies)
        std = np.std(accuracies)

        #mlflow.tensorflow.log_model(model)
        #save_model(model)
        print('round_num:', round_num)
        y_true = 0
        y_pred = 0

        if round_num >= 35:
          y_true, y_pred = previsao(model,
                                    DATASET_VALID_PATH + VIOLENCIA_DIR,
                                    DATASET_VALID_PATH + N_VIOLENCIA_DIR,
                                    round_num)

          try:
            calculate_metrics(y_true, y_pred, round_num)
          except:
            print('não deu para calculate_metrics') 

          try:
            gerar_matriz_confusao(y_true, y_pred, round_num)
          except:
            print('não deu para gerar a matriz de confusao') 

        log_scalar('accuracy', accuracy, round_num)
        log_scalar('Loss', loss, round_num)
        log_scalar('std', std, round_num)


        return loss, { "accuracy": accuracy, "std": std, "y_true": y_true, "y_pred": y_pred, "round_num": round_num }

    return evaluate


def evoluation_dataset():
    train_datagen = ImageDataGenerator(rescale = 1.0/255.)

    return train_datagen.flow_from_directory(DATASET_VALID_PATH,
                                             batch_size=32,
                                             class_mode='categorical',
                                             target_size=TARGET_SIZE)


def fit_config(rnd: int):
    """Return training configuration dict for each round.

    Keep batch size fixed at 32, perform two rounds of training with one
    local epoch, increase to two local epochs afterwards.
    """
    config = {
        "batch_size": 32,
        "local_epochs": 1 if rnd < 2 else 2,
    }
    return config


def evaluate_config(rnd: int):
    """Return evaluation configuration dict for each round.

    Perform five local evaluation steps on each client (i.e., use five
    batches) during rounds one to three, then increase to ten local
    evaluation steps.
    """
    val_steps = 5 if rnd < 4 else 10
    return {"val_steps": val_steps}


def log_scalar(name, value, step):
    """Log a scalar value to both MLflow and TensorBoard"""
    mlflow.log_metric(name, value, step=step)

### Models

In [ ]:
metrics = [
  'accuracy'
]

In [ ]:
def load_inceptionv3():
    local_weights_file = MODELS_PATH + 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = InceptionV3(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('mixed10')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001),
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])

    return model

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

def load_mobilenetv2():
    local_weights_file = MODELS_PATH + 'mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_128_no_top.h5'
    pre_trained_model = MobileNetV2(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('out_relu')
    print('last layer output shape: ', last_layer.output_shape)
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

In [ ]:
from tensorflow.keras.applications.resnet_v2 import ResNet152V2

def load_resnet152v2():
    local_weights_file = MODELS_PATH + 'resnet152v2_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = ResNet152V2(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('post_bn')
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16

def load_vgg16():
    local_weights_file = MODELS_PATH + 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
    pre_trained_model = VGG16(input_shape = (120, 160, 3),
                                    include_top = False,
                                    weights = None)

    pre_trained_model.load_weights(local_weights_file)
    # Make all the layers in the pre-trained model non-trainable
    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('block5_pool')
    last_output = last_layer.output

    # Flatten the output layer to 1 dimension
    x = layers.Flatten()(last_output)
    # Add a fully connected layer with 1,024 hidden units and ReLU activation
    x = layers.Dense(1024, activation='relu')(x)
    # Add a dropout rate of 0.2
    x = layers.Dropout(0.2)(x)
    # Add a final sigmoid layer for classification
    x = layers.Dense(2, activation='softmax')(x)

    model = Model(pre_trained_model.input, x)
    model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='categorical_crossentropy', metrics=metrics)

    return model

### Metrics

In [ ]:

def save_model(model):
    diretorio = SAVE_MODELS_PATH + 'output/final'
    model.save(diretorio)
    converter_tflite = tf.lite.TFLiteConverter.from_saved_model(diretorio)
    tflite_model = converter_tflite.convert()
    with open(diretorio + '/incepv3.tflite', 'wb') as f:
        f.write(tflite_model)

    save_summary(model)


def save_summary(model):
    from contextlib import redirect_stdout
    temp_name = 'modelsummary.txt'
    with open(temp_name, 'w') as f:
        with redirect_stdout(f):
            model.summary()
    mlflow.log_artifact(temp_name, 'Model Summary')


def previsao(modelo, DirPositivo, DirNegativo, round_num):
    ''' Realiza a validacao do modelo
    '''
    ypred = []
    yreal = []
    FP, FN, TP, TN = 0,0,0,0

    for filename in os.listdir(DirPositivo):
        yreal.append(1)
        if "png" in filename or "jpg" in filename:
            file_path = os.path.join(DirPositivo, filename)
            img = image.load_img(file_path, target_size=TARGET_SIZE)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x / 255.
            images = np.vstack([x])
            classes = modelo.predict(images)
            #modelo.predict()
            if classes[0][1]>0.5:
                ypred.append(1)
                TP+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')
            else:
                ypred.append(0)
                FP+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')

    for filename in os.listdir(DirNegativo):
        yreal.append(0)
        if "png" in filename or "jpg" in filename:
            file_path = os.path.join(DirNegativo, filename)
            img = image.load_img(file_path, target_size=TARGET_SIZE)
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = x / 255.

            images = np.vstack([x])
            classes = modelo.predict(images)
            if classes[0][0]<=0.5:
                ypred.append(0)
                TN+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')
            else:
                ypred.append(1)
                FN+=1
                print("\rTP: %i; FP: %i; TN: %i; FN: %i"%(TP,FP,TN,FN), end='')

    log_scalar('TP', TP, round_num)
    log_scalar('FP', FP, round_num)
    log_scalar('TN', TN, round_num)
    log_scalar('FN', FN, round_num)

    return yreal, ypred


def calculate_metrics(y_true, y_predict, round_num):
    acuracia = accuracy_score(y_true, y_predict)
    precision = precision_score(y_true, y_predict)
    recall = recall_score(y_true, y_predict)
    f1 = f1_score(y_true, y_predict)

    print("\nAcurácia: {}\nPrecision: {}\nRecall: {}\nF1-Score: {}".format(acuracia, precision, recall, f1))

    log_scalar('val_accuracy', acuracia, round_num)
    log_scalar('precision', precision, round_num)
    log_scalar('recall', recall, round_num)
    log_scalar('f1', f1, round_num)

def log_scalar(name, value, step):
    """Log a scalar value to both MLflow and TensorBoard"""
    mlflow.log_metric(name, value, step=step)

def gerar_matriz_confusao(y_true, y_pred, round_num):
    matriz_conf = matriz_confusao(y_true, y_pred)
    temp_name = "confusion-matrix-(round-" + str(round_num) + ").png"
    matriz_conf.savefig(temp_name)
    mlflow.log_artifact(temp_name, "confusion-matrix-plots-round-" + str(round_num))

def matriz_confusao(y_true, y_predict):
    matriz_conf = confusion_matrix(y_true, y_predict)
    fig = plt.figure()
    ax = plt.subplot()
    sns.heatmap(matriz_conf, annot=True, cmap='Blues', ax=ax)

    ax.set_xlabel('Valor Predito')
    ax.set_ylabel('Valor Real')
    ax.set_title('Matriz de Confusão')
    ax.xaxis.set_ticklabels(['0', '1'])
    ax.yaxis.set_ticklabels(['0', '1'])
    plt.close()
    return fig


### Main

In [ ]:
#model = load_inceptionv3()
model = load_mobilenetv2()
#model = load_resnet152v2()
#model = load_vgg16()
accuracies = []

last layer output shape:  (None, 4, 5, 1280)


In [ ]:
# Create strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_eval=0.2,
    min_fit_clients=2, #3
    min_eval_clients=2, #3
    min_available_clients=2,
    eval_fn=get_eval_fn(model, accuracies),
    on_fit_config_fn=fit_config,
    on_evaluate_config_fn=evaluate_config,
    initial_parameters=fl.common.weights_to_parameters(model.get_weights()),
)

Found 1252 images belonging to 2 classes.


In [ ]:
# Start Flower server for four rounds of federated learning
mlflow.log_param('round_num', 40)
fl.server.start_server("0.0.0.0:3002", config={"num_rounds": 40}, strategy=strategy)

INFO flower 2022-02-13 23:17:37,084 | app.py:80 | Flower server running (insecure, 40 rounds)
INFO flower 2022-02-13 23:17:37,089 | server.py:118 | Initializing global parameters
INFO flower 2022-02-13 23:17:37,091 | server.py:300 | Using initial parameters provided by strategy
INFO flower 2022-02-13 23:17:37,094 | server.py:120 | Evaluating initial parameters


40/40 [==============================] - 72s 2s/step - loss: 0.9423 - accuracy: 0.5008
round_num: 1


INFO flower 2022-02-13 23:18:54,461 | server.py:127 | initial parameters (loss, other metrics): 0.9423354268074036, {'accuracy': 0.5007987022399902, 'std': 0.0, 'y_true': 0, 'y_pred': 0, 'round_num': 1}
INFO flower 2022-02-13 23:18:54,463 | server.py:133 | FL starting
DEBUG flower 2022-02-13 23:18:54,465 | server.py:255 | fit_round: strategy sampled 2 clients (out of 3)
DEBUG flower 2022-02-13 23:19:39,896 | server.py:264 | fit_round received 2 results and 0 failures


40/40 [==============================] - 48s 1s/step - loss: 2.7765 - accuracy: 0.6717
round_num: 2


INFO flower 2022-02-13 23:21:05,949 | server.py:154 | fit progress: (1, 2.7764687538146973, {'accuracy': 0.6717252135276794, 'std': 0.0854632556438446, 'y_true': 0, 'y_pred': 0, 'round_num': 2}, 131.48486787900038)
INFO flower 2022-02-13 23:21:05,951 | server.py:199 | evaluate_round: no clients selected, cancel
DEBUG flower 2022-02-13 23:21:05,956 | server.py:255 | fit_round: strategy sampled 2 clients (out of 3)
DEBUG flower 2022-02-13 23:21:44,674 | server.py:264 | fit_round received 2 results and 0 failures


...

INFO flower 2022-02-14 00:43:19,756 | server.py:154 | fit progress: (38, 0.010013529099524021, {'accuracy': 0.9968051314353943, 'std': 0.11924922873009138, 'y_true': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

40/40 [==============================] - 49s 1s/step - loss: 0.0105 - accuracy: 0.9968
round_num: 40
TP: 626; FP: 0; TN: 4; FN: 622
Acurácia: 0.5031948881789138
Precision: 0.5016025641025641
Recall: 1.0
F1-Score: 0.6680896478121665


INFO flower 2022-02-14 00:47:40,273 | server.py:154 | fit progress: (39, 0.010507459752261639, {'accuracy': 0.9968051314353943, 'std': 0.11815789852268946, 'y_true': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

40/40 [==============================] - 49s 1s/step - loss: 0.0115 - accuracy: 0.9944
round_num: 41
TP: 623; FP: 3; TN: 4; FN: 622
Acurácia: 0.5007987220447284
Precision: 0.5004016064257029
Recall: 0.9952076677316294
F1-Score: 0.6659540352752539


INFO flower 2022-02-14 00:52:35,306 | server.py:154 | fit progress: (40, 0.01145203784108162, {'accuracy': 0.9944089651107788, 'std': 0.11706149238990028, 'y_true': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
mlflow.end_run()